In [106]:
import pandas as pd
from collections import Counter
from joblib import Parallel, delayed
import multiprocessing
import re
import unicodedata


pd.set_option('display.max_columns', None)

In [42]:

print('⚙️ Importing authors...')
authors = pd.read_csv("./datasets/autores/concat-autores.csv", encoding='iso8859_1', delimiter=";")

# Only selects author 5% sample of the dataset
# authors = authors.sample(frac=0.05, random_state=1)

# authors = pd.concat([authors_2017, authors_2019], axis=0, ignore_index=True)

print("   {} authors in the dataset".format(len(authors)))

⚙️ Importing data...
   145633 authors in the dataset


In [43]:
# Data Type Optimization
# https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2

# def optimize_floats(df: pd.DataFrame) -> pd.DataFrame:
#     floats = df.select_dtypes(include=['float64']).columns.tolist()
#     df[floats] = df[floats].apply(pd.to_numeric, downcast='integer')
#     return df

# def optimize_ints(df: pd.DataFrame) -> pd.DataFrame:
#     ints = df.select_dtypes(include=['int64']).columns.tolist()
#     df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
#     return df

# def optimize_objects(df: pd.DataFrame) -> pd.DataFrame:
#     for col in df.select_dtypes(include=['object']):
#         if not (type(df[col][0])==list):
#             num_unique_values = len(df[col].unique())
#             num_total_values = len(df[col])
#             if float(num_unique_values) / num_total_values < 0.5:
#                 df[col] = df[col].astype('category')
#     return df

# def optimize(df: pd.DataFrame):
#     return optimize_floats(optimize_ints(optimize_objects(df)))

# print('⚙️ Optimizing columns data types...')

# optimize(authors)

# authors.info()

In [44]:
# Filter fields of interest
authors = authors.filter([
  'AN_BASE',
  'NM_AUTOR',
  'NM_ABNT_AUTOR',
  'TP_AUTOR',
  'NM_TP_CATEGORIA_DOCENTE',
  'NM_NIVEL_DISCENTE',
  'CD_PROGRAMA_IES',
  'NM_PROGRAMA_IES',
  'NM_AREA_CONHECIMENTO',
  'SG_ENTIDADE_ENSINO',
  'ID_PESSOA_DISCENTE',
  'ID_PESSOA_DOCENTE',
  'ID_PESSOA_PART_EXTERNO',
  'ID_PESSOA_POS_DOC',
  'ID_PESSOA_EGRESSO',
  'ID_ADD_PRODUCAO_INTELECTUAL',
])

# Unify IDs
def unify_ids(cols):
    return {
      'DOCENTE': cols['ID_PESSOA_DOCENTE'],
      'EGRESSO': cols['ID_PESSOA_EGRESSO'],
      'PÓS-DOC': cols['ID_PESSOA_POS_DOC'],
      'DISCENTE': cols['ID_PESSOA_DISCENTE'],
      'PARTICIPANTE EXTERNO': cols['ID_PESSOA_PART_EXTERNO'],
      '-': None,
    }[cols['TP_AUTOR']]

ids = [
  'ID_PESSOA_DISCENTE',
  'ID_PESSOA_DOCENTE',
  'ID_PESSOA_PART_EXTERNO',
  'ID_PESSOA_POS_DOC',
  'ID_PESSOA_EGRESSO',
]

print('⚙️ Unifying author IDs...')

authors['ID'] = authors[['TP_AUTOR', *ids]].apply(unify_ids, axis=1)
authors = authors.drop(columns=ids)

⚙️ Unifying author IDs...


In [112]:
# https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string
def strip_accents(text):
  text = unicodedata.normalize('NFD', text)
  text = text.encode('ascii', 'ignore')
  text = text.decode("utf-8")
  return str(text)

def normalize_name(name):
  if ',' not in name:
    norm = name
  else:
    it = name.split(', ')
    it.reverse()
    norm = ' '.join(it)

  # remove accents
  norm = strip_accents(norm)

  # remove invalid chars
  norm = re.sub('[_-]', ' ', norm)
  norm = re.sub('[0-9?&#;()]', '', norm)

  # remove leading and trailing spaces
  norm = norm.strip()

  return norm

print('⚙️ Normalizing authors names...')
authors['NM_AUTOR'] = authors['NM_AUTOR'].apply(normalize_name)

⚙️ Normalizing authors names...


,AN_BASE,NM_AUTOR,NM_ABNT_AUTOR,TP_AUTOR,NM_TP_CATEGORIA_DOCENTE,NM_NIVEL_DISCENTE,CD_PROGRAMA_IES,NM_PROGRAMA_IES,NM_AREA_CONHECIMENTO,SG_ENTIDADE_ENSINO,ID_ADD_PRODUCAO_INTELECTUAL,ID,FIRST_LAST_NAME,FULL_NAME
1037671,2018,RENATO DE MELLO PRADO,"PRADO, RENATO DE MELLO",DOCENTE,PERMANENTE,NaN,33004102071P2,AGRONOMIA (CIÊNCIA DO SOLO),NaN,UNESP-JAB,24309943,10435.0,RENATO PRADO,RENATO DE MELLO PRADO
747777,2017,JULIANE NICOLODI CAMERA,"CAMERA, J. N.",PARTICIPANTE EXTERNO,NaN,NaN,42037018001P9,DESENVOLVIMENTO RURAL,NaN,UNICRUZ,23597579,81330.0,JULIANE CAMERA,JULIANE NICOLODI CAMERA
2828750,2019,CRISTIANE OLIVEIRA ALVES TELLES,"TELLES, C. O. A.",-,NaN,NaN,31004016013P1,PSICOLOGIA SOCIAL,PSICOLOGIA,UERJ,26613923,NaN,CRISTIANE TELLES,CRISTIANE OLIVEIRA ALVES TELLES
1403413,2018,THAIZA LUIZA COSTA SANTOS,"SANTOS, THAIZA LUIZA COSTA",-,NaN,NaN,28001010019P5,ARQUITETURA E URBANISMO,ARQUITETURA E URBANISMO,UFBA,25016379,NaN,THAIZA SANTOS,THAIZA LUIZA COSTA SANTOS
1209322,2018,JOSE MANSUR ASSAF,"ASSAF, J. M.",DOCENTE,PERMANENTE,NaN,33001014006P1,ENGENHARIA QUÍMICA,NaN,UFSCAR,24664812,24220.0,JOSE ASSAF,JOSE MANSUR ASSAF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929815,2018,CARINE CARLA CAREZIA,"CAREZIA, C. C.",DISCENTE,NaN,BACHARELADO,41020014004P9,CIÊNCIA E TECNOLOGIA AMBIENTAL,NaN,UFFS,24073780,3307435.0,CARINE CAREZIA,CARINE CARLA CAREZIA
2624701,2018,JULIO CESAR CRUZ C DA ROCHA,"COLLARES-DA-ROCHA, J. C. C.",DOCENTE,PERMANENTE,NaN,31019013003P0,PSICOLOGIA,NaN,UCP/RJ,25029197,84411.0,JULIO ROCHA,JULIO CESAR CRUZ C DA ROCHA
1353928,2018,GILDA SOUSA DE ALVARENGA,"ALVARENGA, G. S.",PARTICIPANTE EXTERNO,NaN,NaN,31003010045P8,SISTEMAS DE GESTÃO,NaN,UFF,24924948,2258464.0,GILDA ALVARENGA,GILDA SOUSA DE ALVARENGA
2431163,2017,ISLANY COSTA ALENCAR,"ALENCAR, I. C.",DISCENTE,NaN,MESTRADO,24001015001P4,EDUCAÇÃO,NaN,UFPB-JP,23541968,2384606.0,ISLANY ALENCAR,ISLANY COSTA ALENCAR


In [46]:
def firstAndLastName(name):
  it = name.split(' ')
  return ' '.join([it[0], it[-1]])

print('⚙️ Creating helper columns...')
authors['FIRST_LAST_NAME'] = authors['NM_AUTOR'].apply(firstAndLastName)
authors['FULL_NAME'] = authors['NM_AUTOR']

⚙️ Creating helper columns...


In [47]:
## Helper Functions
def agg_to_dict(items):
  return items.astype(str).value_counts().to_dict()

def most_frequent(items):
  values = items.dropna()
  if len(values) == 0: return None
  occurence_count = Counter(values)
  return occurence_count.most_common(1)[0][0]

def toArray(item):
  return item if hasattr(item, '__iter__') and not isinstance(item, str) else [item]

def priority(priority_list):
  return lambda items: next((type for type in priority_list if type in items.array), None)

merge_schema = {
  'FULL_NAME': 'first',
  'NM_AUTOR': agg_to_dict,
  'NM_ABNT_AUTOR': agg_to_dict,
  'FIRST_LAST_NAME': agg_to_dict,
  'TP_AUTOR': agg_to_dict,
  'NM_TP_CATEGORIA_DOCENTE': agg_to_dict,
  'NM_NIVEL_DISCENTE': agg_to_dict,
  'CD_PROGRAMA_IES': agg_to_dict,
  'NM_PROGRAMA_IES': agg_to_dict,
  'NM_AREA_CONHECIMENTO': agg_to_dict,
  'SG_ENTIDADE_ENSINO': agg_to_dict,
  'ID_ADD_PRODUCAO_INTELECTUAL': list,
}

# 'NM_AUTOR': most_frequent,
# 'NM_ABNT_AUTOR': most_frequent,
# 'TP_AUTOR': higher_priority(['DOCENTE', 'EGRESSO', 'PÓS-DOC', 'DISCENTE', 'PARTICIPANTE EXTERNO']),
# 'NM_TP_CATEGORIA_DOCENTE': higher_priority(['PERMANENTE', 'COLABORADOR', 'VISITANTE']),
# 'NM_NIVEL_DISCENTE': higher_priority(['DOUTORADO PROFISSIONAL', 'BACHARELADO', 'MESTRADO', 'DOUTORADO', 'MESTRADO PROFISSIONAL', ]),
# 'NM_PROGRAMA_IES': most_frequent,
# 'NM_AREA_CONHECIMENTO': most_frequent,
# 'SG_ENTIDADE_ENSINO': most_frequent,
# 'ID_ADD_PRODUCAO_INTELECTUAL': list,

In [48]:
# Merge authors by ID
print('⚙️ Merging authors by ID...')
merged_authors = authors.groupby(['ID'], sort=False, as_index=False).agg(merge_schema)
print("   {} authors with ID after merge".format(len(merged_authors)))

merged_authors

⚙️ Merging authors by ID...
   89144 authors with ID after merge


,ID,FULL_NAME,NM_AUTOR,NM_ABNT_AUTOR,FIRST_LAST_NAME,TP_AUTOR,NM_TP_CATEGORIA_DOCENTE,NM_NIVEL_DISCENTE,CD_PROGRAMA_IES,NM_PROGRAMA_IES,NM_AREA_CONHECIMENTO,SG_ENTIDADE_ENSINO,ID_ADD_PRODUCAO_INTELECTUAL
0,10435.0,RENATO DE MELLO PRADO,{'RENATO DE MELLO PRADO': 14},"{'PRADO, RENATO DE MELLO': 9, 'PRADO, R. M.': 5}",{'RENATO PRADO': 14},"{'DOCENTE': 11, 'PARTICIPANTE EXTERNO': 3}","{'PERMANENTE': 11, 'nan': 3}",{'nan': 14},"{'33004102071P2': 8, '51001012034P6': 3, '3300...","{'AGRONOMIA (CIÊNCIA DO SOLO)': 8, 'AGRONOMIA'...",{'nan': 14},"{'UNESP-JAB': 11, 'UFMS': 3}","[24309943, 24309891, 22718287, 24309813, 26695..."
1,81330.0,JULIANE NICOLODI CAMERA,{'JULIANE NICOLODI CAMERA': 4},"{'CAMERA, J. N.': 4}",{'JULIANE CAMERA': 4},{'PARTICIPANTE EXTERNO': 4},{'nan': 4},{'nan': 4},"{'42037018001P9': 2, '42037018002P5': 2}","{'DESENVOLVIMENTO RURAL': 2, 'PRÁTICAS SOCIOCU...",{'nan': 4},{'UNICRUZ': 4},"[23597579, 23597874, 23597527, 23597772]"
2,24220.0,JOSE MANSUR ASSAF,{'JOSE MANSUR ASSAF': 2},"{'ASSAF, J. M.': 2}",{'JOSE ASSAF': 2},{'DOCENTE': 2},{'PERMANENTE': 2},{'nan': 2},{'33001014006P1': 2},{'ENGENHARIA QUÍMICA': 2},{'nan': 2},{'UFSCAR': 2},"[24664812, 23326964]"
3,3080064.0,ANGELIA PACHECO DOS SANTOS,{'ANGELIA PACHECO DOS SANTOS': 1},"{'SANTOS, A. P.': 1}",{'ANGELIA SANTOS': 1},{'DISCENTE': 1},{'nan': 1},{'BACHARELADO': 1},{'41002016158P5': 1},{'CIÊNCIAS AMBIENTAIS': 1},{'nan': 1},{'UDESC': 1},[23251852]
4,13996.0,VLADIMIR JESUS TRAVA AIROLDI,{'VLADIMIR JESUS TRAVA AIROLDI': 3},"{'AIROLDI, V. J. T.': 2, 'TRAVA-AIROLD, V. J.'...",{'VLADIMIR AIROLDI': 3},"{'DOCENTE': 2, 'PARTICIPANTE EXTERNO': 1}","{'COLABORADOR': 1, 'nan': 1, 'PERMANENTE': 1}",{'nan': 3},"{'33010013009P6': 1, '33009015172P0': 1, '3300...","{'ENGENHARIA E TECNOLOGIA ESPACIAIS': 1, 'ENGE...",{'nan': 3},"{'UNIFESP': 2, 'INPE': 1}","[25158104, 25802031, 22158755]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89139,1346444.0,GUY LOUARN,{'GUY LOUARN': 1},"{'LOUARN, G.': 1}",{'GUY LOUARN': 1},{'PARTICIPANTE EXTERNO': 1},{'nan': 1},{'nan': 1},{'33009015075P4': 1},{'QUÍMICA - CIÊNCIA E TECNOLOGIA DA SUSTENTABI...,{'nan': 1},{'UNIFESP': 1},[23888159]
89140,6253.0,MARIA MEDIANEIRA PADOIN,{'MARIA MEDIANEIRA PADOIN': 1},"{'PADOIN, M. M.': 1}",{'MARIA PADOIN': 1},{'DOCENTE': 1},{'PERMANENTE': 1},{'nan': 1},{'42002010051P2': 1},{'HISTÓRIA': 1},{'nan': 1},{'UFSM': 1},[25147188]
89141,47078.0,RICARDO TREZZI CASA,{'RICARDO TREZZI CASA': 1},"{'CASA, R. T.': 1}",{'RICARDO CASA': 1},{'DOCENTE': 1},{'PERMANENTE': 1},{'nan': 1},{'41002016007P7': 1},{'PRODUÇÃO VEGETAL': 1},{'nan': 1},{'UDESC': 1},[22305181]
89142,368415.0,ORLEY OLAVO FILEMON,{'ORLEY OLAVO FILEMON': 1},"{'FILEMON, O. O.': 1}",{'ORLEY FILEMON': 1},{'DISCENTE': 1},{'nan': 1},{'DOUTORADO': 1},{'52002012004P9': 1},{'EDUCAÇÃO': 1},{'nan': 1},{'PUC-GOIÁS': 1},[24793413]


In [93]:
# Get all the authors without an ID (orphan authors)
orphan_authors = authors[authors['ID'].isnull()]
print("   {} authors without IDs".format(len(orphan_authors)))

   16004 authors without IDs


In [104]:
# Compare two authors and return a value `n` indicating the probability that both authors are the same person
def compare_authors(author, orphan):
  n = 0

  # Exact name match
  if orphan['FULL_NAME'] in author['NM_AUTOR']: n = n + 5;
  # Match first and last name
  if orphan['FIRST_LAST_NAME'] in author['FIRST_LAST_NAME']: n = n + 2;

  # Return if there's no chance of match
  if n == 0: return 0

  # Match abnt name
  if orphan['NM_ABNT_AUTOR'] in author['NM_ABNT_AUTOR']: n = n + 1;
  # Match university
  if orphan['SG_ENTIDADE_ENSINO'] in author['SG_ENTIDADE_ENSINO']: n = n + 1;
  # Match author type
  if orphan['TP_AUTOR'] in author['TP_AUTOR']: n = n + 1;
  # Match IES program
  if orphan['CD_PROGRAMA_IES'] in author['CD_PROGRAMA_IES']: n = n + 1;

  return n

def update_item_count(item_count, value):
  item_count[value] = 1 if value not in item_count else item_count[value] + 1

def merge_authors(author, orphan_author):
  merged = author.copy(deep=True)
  for column in author.index.to_list():
    if column not in merge_schema or column == 'FULL_NAME': continue
    author_value = author[column]
    orphan_value = orphan_author[column]
    if isinstance(author_value, list):
      author_value.append(orphan_value)
    else:
      update_item_count(author_value, orphan_value)

  return merged

# Merging orphan authors
merge_count = 0
append_count = 0

p_authors = merged_authors

def process_null_author(idx_na):
  global merged_authors, p_authors, merge_count, append_count
  
  try:
    orphan = orphan_authors.iloc[idx_na]
    last_name = orphan['FULL_NAME'].split(' ')[-1]
    potential_authors = merged_authors[merged_authors['FULL_NAME'].str.contains(last_name, na=False)]
    
    for idx_pot in range(len(potential_authors)):
      author = potential_authors.iloc[idx_pot]
      
      if compare_authors(author, orphan) >= 5:
        print("   🔄 Merging authors ({})'{}' to ({})'{}'".format(idx_na, orphan['FULL_NAME'], idx_pot, author['FULL_NAME']))
        merged = merge_authors(author, orphan)
        p_authors.loc[[author.name]] = pd.DataFrame(merged)
        merge_count = merge_count + 1
        return
    print("   Appending author ({})'{}'".format(idx_na, orphan['NM_AUTOR']))
    orphan_df = pd.DataFrame(orphan).T.groupby(['NM_AUTOR']).agg(merge_schema)
    p_authors = pd.concat([p_authors, orphan_df], ignore_index=True)
    append_count = append_count + 1
  except:
    print("   Error processing author ({})'{}', '{}'  -- skipping".format(idx_na, orphan['NM_AUTOR'], orphan['NM_ABNT_AUTOR']))
    pass

num_cores = multiprocessing.cpu_count()

print('⚙️ Merging authors without IDs (using {} cores):'.format(num_cores))

# Single thread processing (keep commented)
# for i in range(len(orphan_authors)): process_null_author(i)

# Parallel processing
Parallel(n_jobs=num_cores, require='sharedmem')(delayed(process_null_author)(i) for i in range(len(orphan_authors)))

print("   {} authors were merged and {} were appended to the dataset.".format(merge_count, append_count))

⚙️ Merging authors without IDs (using 16 cores):
   Appending author (6)'HANS COZIJN'
   Appending author (0)'CRISTIANE OLIVEIRA ALVES TELLES'
   Appending author (4)'ROGÉLIO MORENO SANTISTEBAN'
   Appending author (2)'JOSYMEIRE BARROS FRAZÃO'
   Appending author (11)'GABRIELA DELGADO SORIANO'   Appending author (15)'TAINÁ PIGOSSO'

   Appending author (16)'ELIANE MAYUMI KATO-NARITA'
   Appending author (18)'GABRIEL WAINER'
   Appending author (20)'DEIVISON HENRIQUE TEIXEIRA FIRMO'
   Appending author (22)'ANDRÉ DANTAS GERMANO'
   Appending author (19)'CAROLAYNE MABEL VICTOR DA CUNHA'   🔄 Merging authors (21)'ALINE TELLES BIASOTO MARQUES' to (136)'ALINE TELLES BIASOTO MARQUES'

   Appending author (24)'LEO A. S. AGBETOYE'
   Appending author (3)'SAULO A ARAUJO'
   Appending author (14)'RUTH BEZERRA RODRIGUES'
   Appending author (23)'NATALIA DELAVIE MENDES'
   Appending author (29)'SANDRA BORSOI'
   Appending author (12)'MAGDA GOMES DA SILVA COSTA'   Appending author (8)'GABRIELA CARDO

KeyboardInterrupt: 

   Appending author (188)'CARINE COUTINHO DA SILVA'


In [ ]:
print('⚙️ Exporting authors to processed_authors.csv...')

p_authors.to_csv('processed_authors.csv')

In [ ]:
# idx_a = 21335 # 79797, 21335, 171
# idx_na = 2 # 0, 2, 13
# merged_authors.iloc[[idx_a]] = merge_authors(merged_authors.iloc[[idx_a]], orphan_authors.iloc[[idx_na]])

In [ ]:
# a = merged_authors.loc[[58693]]
# b = orphan_authors.loc[[1051112]]

# display(a)
# display(b)

# m = merge_authors(a, b)

# display(m)

In [ ]:
# merged_authors = pd.concat([merged_authors, orphan_authors], ignore_index=True)
# merged_authors

In [ ]:
# merged_authors.to_excel('processed_authors.xlsx')

In [ ]:
# def merge_authors(author, orphan_author):
#   merged = author.copy(deep=True)
#   for column in author.index.to_list():
#     print(column)
#     if column not in merge_schema: continue
#     agg_func = merge_schema[column]
#     itemA = toArray(author.get(column).iloc[0])
#     itemB = toArray(orphan_author.get(column).iloc[0])
#     value = agg_func(pd.Series([*itemA, *itemB]))
#     merged[column] = [value]
#   return merged

In [ ]:
merged_authors[merged_authors['NM_AUTOR'].str.contains("RECAMONDE", na=False)]